In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tanzilhussain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
from transformers import pipeline
import torch 

from nltk import word_tokenize, pos_tag
import textstat
from pathlib import Path
import json

In [7]:
# goal: sentiment- detect emotional warmth, empathy, tone
sentiment_pipe = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")


Device set to use mps:0


In [15]:
label_score ={
    'LABEL_0': 0.0,  # negative
    'LABEL_1': 0.0,  # neutral
    'LABEL_2': 0.0,  # positive
}

def get_sentiment_score(text):
    try:
        result = sentiment_pipe(text[:512][0])
        # get the assigned label or default to .5
        return label_score.get(result['label'], 0.5)
    except Exception as e:
        print("error with sentiment score")
        return 0.5

# goal: readability- check if text is clearly written
def get_readability_score(text):
    result = textstat.flesch_reading_ease(text) # from 0-100, good is 70-80
    result = max(0, min(result/100,1)) # normalize
    return result

# goal: specificity- how detailed or vague is the text?
def get_specificity_score(text):
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens) # assigns part of speech to each tag
    detail_words = [word for word, tag in tagged if tag in ("NN", "NNS", "NNP", "VB", "VBD", "VBN", "VBG")] # only flags nouns and words bc those contribute to specificity the most
    return len(detail_words) / len(tokens)

# goal: personalization- how personal is the advice
def get_personalization_score(text):
    personal_words = ["you", "your", "yours", "i", "me", "my", "mine", "we", "us", "our"]
    tokens = word_tokenize(text.lower())
    for token in tokens:
        count = sum(1 for word in tokens if word in personal_words)


def total_trust_score(text):
    total = (.25 * get_personalization_score(text)) + (.25 * get_specificity_score(text)) +(.25 * get_readability_score(text)) + (.25 * get_sentiment_score(text))
    return total


In [14]:
output_path = Path("data/dear_abby_scored.jsonl")
data = []
with output_path.open("r") as file:
    for line in file:
        if line.strip():
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line: {line.strip()}\nError: {e}")

for line in data:
    text = line["ai_advice"]
    print(get_personalization_score(text))
#     total = total_trust_score(line["ai_advice"])
#     line["trust_score"] = total

# with output_path.open("w") as f:
#     for line in data:
#         f.write(json.dumps(line) + "\n")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/tanzilhussain/nltk_data'
    - '/Users/tanzilhussain/ai-trust-score/.venv/nltk_data'
    - '/Users/tanzilhussain/ai-trust-score/.venv/share/nltk_data'
    - '/Users/tanzilhussain/ai-trust-score/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
